In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import eleanor
from astropy.timeseries import BoxLeastSquares
import pandas as pd
from scipy.signal import savgol_filter

from lightkurve import LightCurve
from lightkurve import periodogram

%matplotlib inline

In [ ]:
#fn_ = "hlsp_eleanor_tess_ffi_tic%i_s0%i_tess_v0.1.8_lc.fits" % (tic_[ii], sector)

#file_exists_ = os.path.exists("/Users/vkostov/.eleanor/" + fn_)    
    
#if file_exists_ == False:
#    star = eleanor.multi_sectors(tic=tic_, sectors=sectors_, tc=True)
#else:

In [ ]:
sectors_ = [7]#[1,2,3,4,5,6,7,8,9,10]#[1,2,3,4,5,6,7,8,9,10]
tic_ = 281341249#70763084#35655828#
star = eleanor.multi_sectors(tic=tic_, sectors=sectors_, tc=True)

plt.figure(figsize=(15,5))

time_eleanor = np.array([])
flux_eleanor = np.array([])
time_sector =[]
flux_sector = []
#plt.figure(figsize=(15,5))
for i in range(len(star)):
#    print(i)
    data0 = eleanor.TargetData(star[i], height=15, width=15, bkg_size=31, do_psf=True, do_pca=False)

    q0 = data0.quality == 0

    plt.plot(data0.time[q0], data0.psf_flux[q0]/np.nanmedian(data0.psf_flux[q0]) + 0.00, 'k.-')
    
    time_eleanor = np.append(time_eleanor, data0.time[q0])
    flux_eleanor = np.append(flux_eleanor, data0.psf_flux[q0]/np.median(data0.psf_flux[q0]))
    time_sector = np.append(time_sector, [time_eleanor])
    flux_sector = np.append(flux_sector, [flux_eleanor])

plt.ylabel('Normalized Flux', fontsize=24)
plt.xlabel('Time', fontsize=24)
plt.yticks(fontsize=18)
plt.xticks(fontsize=18)
plt.ylim(0.90, 1.03)

plt.show()

In [ ]:
import csv

save_file_ = 'n'

if save_file_ == 'y':
    with open(str(tic_) + '_Eleanor_LC_S1to10', mode = 'w') as txt:
        txt_w = csv.writer(txt, delimiter = ' ')
        for time, flux in zip(time_eleanor, flux_eleanor):
            txt_w.writerow([time, flux])

In [ ]:
tt_r, ff_r = data0.time[q0], data0.raw_flux[q0]/np.nanmedian(data0.raw_flux[q0])
tt_p, ff_p = data0.time[q0], data0.psf_flux[q0]/np.nanmedian(data0.psf_flux[q0])
plt.figure(figsize=(15,5))
plt.plot(tt_r, ff_r, 'k.-')
plt.plot(tt_p, ff_p, 'r.-')
nn_ = 0
#plt.xlim(tt_r[0]+nn_*5, tt_r[0]+5+nn_*5)
#plt.ylim(0.7, 1.03)
plt.ylim(0.95, 1.04)

In [ ]:
plt.plot(time_eleanor_tmp, flux_eleanor_tmp)

In [ ]:
import lightkurve as lk
pixels = lk.search_targetpixelfile("TIC " + str(tic)).download()

In [ ]:
lcTIC = LightCurve(time_eleanor, flux_eleanor)

b=lcTIC.remove_outliers(sigma=5).flatten().to_pandas()

time_detrend_ = np.asarray(b.time)
flux_detrend_ = np.asarray(b.flux)

t0 = min(time_detrend_)
time_detrend_-= min(time_detrend_)

In [ ]:
#for ii in range(len(time_detrend_)):
#    print(t0+time_detrend_[ii], flux_detrend_[ii])

In [ ]:
m = np.ones(len(flux_detrend_), dtype=bool)

#m = (time_detrend_ > 77.) & (time_detrend_ < 106.)
#m = (time_detrend_ > 145.) & (time_detrend_ < 150.)
#m = (time_detrend_ > 195.) & (time_detrend_ < 200.)
#
#m[(time_detrend_ > 67.) & (time_detrend_ < 70.)] = False
#m[(time_detrend_ > 85.) & (time_detrend_ < 88.)] = False
#m[(time_detrend_ > 155.) & (time_detrend_ < 162.)] = False

#m[(time_detrend_ > 27.) & (time_detrend_ < 30.)] = False
#m[(time_detrend_ > 42.) & (time_detrend_ < 45.)] = False
#m[(time_detrend_ > 110.) & (time_detrend_ < 115.)] = False
#m[(time_detrend_ > 165.) & (time_detrend_ < 167.)] = False
#m[(time_detrend_ > 210.) & (time_detrend_ < 212.)] = False
#m[(time_detrend_ > 267.) & (time_detrend_ < 275.)] = False
#m[(time_detrend_ > 242.) & (time_detrend_ < 245.)] = False

time_detrend_ = time_detrend_[m]
flux_detrend_ = flux_detrend_[m]

In [ ]:
fig = plt.subplots(figsize=(20, 5))
plt.plot(time_detrend_, flux_detrend_, 'ko-')
#plt.xlim(200., 250.)

In [ ]:
# DETREND DATA

def detrend_data_(time, flux, do_plot_, sector):
    x = time
    y = flux/np.nanmedian(flux)
    mu = np.median(y)
    y = (y / mu - 1)# * 1e3

# Identify outliers
    m = np.ones(len(y), dtype=bool)
    
    for i in range(10):
        y_prime = np.interp(x, x[m], y[m])
        smooth = savgol_filter(y_prime, 21, polyorder=3)
        resid = y - smooth
        sigma = np.sqrt(np.mean(resid**2))
        m0 = np.abs(resid) < 3*sigma
        if m.sum() == m0.sum():
            m = m0
            break
        m = m0
    
    m = (resid < 15*sigma) & (resid > -15*sigma)
#
# Shift the data so that it starts at t=0. This tends to make the fit
# better behaved since t0 covaries with period.
#    print(min(x), min(x[m]))
    x_ref = np.min(x[m]) if sector != 3 else np.min(x)
#    x -= x_ref

# Make sure that the data type is consistent
    x_bak_, y_bak_ = x, y
    x = np.ascontiguousarray(x[m], dtype=np.float64)
    y = np.ascontiguousarray(y[m], dtype=np.float64)
    smooth = np.ascontiguousarray(smooth[m], dtype=np.float64)

    x_detrend_ = x
    y_detrend_ = y - smooth
    
#    do_plot_= 'y'
    if do_plot_ == 'y':
        plt.figure(figsize=(20,5))
    
# Plot the data
#        plt.subplot(211)
        plt.plot(x_bak_+0*x_ref, y_bak_, "k", label="raw data")
        plt.plot(x+0*x_ref, smooth, lw = 3., label = 'sav-gol filter')
        plt.plot(x_bak_[~m]+0*x_ref, y_bak_[~m], "xr", label="outliers")
        plt.legend(fontsize=12)
        plt.xlim(x_bak_.min(), x_bak_.max())
        plt.yticks([])
        plt.xlabel("time")
#        plt.ylabel("flux")
#        plt.ylim(-25., 25.)
        
#        plt.subplot(212)
#        plt.plot(x_detrend_, y_detrend_, "k", label="data")
#        plt.legend(fontsize=12)
#        plt.xlim(x_detrend_.min(), x_detrend_.max())
#        plt.xlabel("time")
#        plt.ylabel("flux")
        plt.show()
    
    return x_detrend_, y_detrend_, x_ref, x_bak_, y_bak_, smooth, m

In [ ]:
#RUN BLS
def run_bls_(x, y, tic, do_plot_, min_period, max_period_):
        
    x_ref = min(x)
    x -= min(x)
    
    period_grid = np.exp(np.linspace(np.log(min_period), np.log(max_period_), 50000))
    durations_ = 0.1+0.05*np.linspace(0,3,4)#0.02+0.02*np.linspace(0,11,12)#0.02+0.02*np.linspace(0,11,12)
    bls_results = []
    periods = []
    t0s = []
    depths = []
    durations = []

    # Compute the periodogram for each planet by iteratively masking out
    # transits from the higher signal to noise planets. Here we're assuming
    # that we know that there are exactly two planets.
    for i in range(1):
        bls = BoxLeastSquares(x, y)
        bls_power = bls.power(period_grid, durations_, oversample=20)
        bls_results.append(bls_power)
    
    # Save the highest peak as the planet candidate
        index = np.argmax(bls_power.power)
        periods.append(bls_power.period[index])
        t0s.append(bls_power.transit_time[index])
        depths.append(bls_power.depth[index])
        durations.append(bls_power.duration[index])
        
    # Mask the data points that are in transit for this candidate
        m = np.zeros(len(x), dtype=bool)
        m |= bls.transit_mask(x, periods[-1], 0.5, t0s[-1])
        
    # PLOT RESULTS   

    if do_plot_ == 'y':

        fig, axes = plt.subplots(len(bls_results), 2, figsize=(20, 10))
        
        for i in range(len(bls_results)):
    # Plot the lightcurve
            plt.subplot(2,3,(1,3))
            plt.plot(x+x_ref, y, 'k-')
            plt.title('detrended TESS lightcurve')
#            plt.ylabel('[ppt]')
            plt.xlim((x + x_ref).min(), (x + x_ref).max())
            plt.yticks([])
    # Plot the periodogram 
            plt.subplot(234)
            plt.plot(np.log10(bls_results[i].period), bls_results[i].power, "k")
#            plt.title('BLS periodogram')
            plt.yticks([])
    # Plot the folded transit (folded on the full period)            
            plt.subplot(235)
            p = periods[i]
            x_fold = (x - t0s[i] + 0.5*p) % p - 0.5*p
            m = np.abs(x_fold) < p
            plt.plot(x_fold[m], y[m], ".k")  
            inds = np.argsort(x_fold[m])
    # Overplot the box           
            plt.plot(x_fold[m][inds], bls.model(x, p, durations[i], t0s[i])[m][inds], 'C1',lw = 2)
#            plt.ylim(-10.*depths[0],5*depths[0])
            plt.xlabel('Phase [days]')
#
#
    # Plot the folded transit (folded on 1-day)   
            plt.subplot(236)
            p = periods[i]
            x_fold = (x - t0s[i] + 0.5*p) % p - 0.5*p
            m = np.abs(x_fold) < 1.0 
            plt.plot(x_fold[m], y[m], ".k")  
            inds = np.argsort(x_fold[m])
    # Overplot the box           
            plt.plot(x_fold[m][inds], bls.model(x, p, durations[i], t0s[i])[m][inds], 'C1',lw = 2)
    # Overplot the phase binned light curve
            bins = np.linspace(-1., 1., 32)#
            denom, _ = np.histogram(x_fold, bins)
            num, _ = np.histogram(x_fold, bins, weights=y)
            denom[num == 0] = 1.0
#            plt.plot(0.5*(bins[1:] + bins[:-1]), num / denom, color="c", lw = 1)
            plt.title('TIC ' + str(tic) + "; P =" + str(np.round(p,2)) + "; to =" + str(round(x_ref+t0s[0],2)))
            plt.xlabel('Phase [days]')
#            plt.ylim(-10.*depths[0],5*depths[0])
        
            plt.show()
#        plt.savefig("/Users/vkostov/Desktop/Ideas_etc/TESS/tesseract/ryan_LCs/pngs/" + str(tic) + ".png")
#        plt.close()

    return bls_power

In [ ]:
flags = np.isfinite(ff_r)

detrend_ = 'y'

if detrend_ == 'y':
    do_plot_ = 'y'
    time_detrend_, flux_detrend_, x_ref, x_bak_, y_bak_, smooth, m = \
    detrend_data_(tt_r[flags], ff_r[flags], do_plot_, sectors_)
else:
    time_detrend_, flux_detrend_ = tt_r[flags], 1e3*ff_r[flags]

do_plot_ = 'y'
min_period = 1.
max_period = int((0.8*(max(time_detrend_) - min(time_detrend_))))
bls_results = run_bls_(time_detrend_, flux_detrend_, tic_, do_plot_, min_period, max_period)

In [ ]:
index = np.argmax(bls_results.power)

per_ = bls_results.period[index]

per_

In [ ]:
nn = 0

time_detrend_, flux_detrend_ = data0.time[q0], data0.psf_flux[q0]/np.nanmedian(data0.psf_flux[q0])

per_ = 9.64
t0 = time_detrend_[0]



n_plots_ = int(max(time_detrend_)/per_)
n_plots_ = 3
fig = plt.subplots(figsize=(20, 20))
for ii in range(n_plots_):

    plt.subplot(n_plots_,1,ii+1)
    
    plt.plot(time_detrend_, flux_detrend_, 'ko-')

    plt.xlim(t0+(ii-nn)*per_,t0+(ii+nn+1)*per_)
#    plt.ylim(0.99,1.005)
    
#plt.savefig('tmp_.png')

In [ ]:
tt_, ff_ = time_detrend_, flux_detrend_

period_grid = np.exp(np.linspace(np.log(1.), np.log(100), 50000))
duration = 0.1+0.05*np.linspace(0,3,4)

bls_periods = np.zeros([4])
bls_t0s = np.zeros([4])
bls_depths = np.zeros([4])
bls_durations = np.zeros([4])

# planet 1
bls = BoxLeastSquares(tt_, ff_)
bls_power = bls.power(period_grid, duration, oversample=20)

bls_power0 = bls_power

# Save the highest peak as the planet candidate
index = np.argmax(bls_power.power)
bls_period = bls_power.period[index]
bls_t0 = bls_power.transit_time[index]
bls_depth = bls_power.depth[index]
bls_duration = bls_power.duration[index]
transit_mask = bls.transit_mask(tt_, bls_period, 3*bls_duration, bls_t0)

bls_periods[0] = bls_period
bls_t0s[0] = bls_t0
bls_depths[0] = bls_depth
bls_durations[0] = bls_duration

# planet 2
bls = BoxLeastSquares(tt_[~transit_mask], ff_[~transit_mask])
bls_power = bls.power(period_grid, duration, oversample=20)

# Save the highest peak as the planet candidate
index = np.argmax(bls_power.power)
bls_period = bls_power.period[index]
bls_t0 = bls_power.transit_time[index]
bls_depth = bls_power.depth[index]
bls_duration = bls_power.duration[index]
transit_mask = bls.transit_mask(tt_, bls_period, 3*bls_duration, bls_t0)

bls_periods[1] = bls_period
bls_t0s[1] = bls_t0
bls_depths[1] = bls_depth
bls_durations[1] = bls_duration

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 10))

# Plot the periodogram
ax = axes[0]
ax.axvline(np.log10(bls_periods[0]), color="C1", lw=5, alpha=0.8)
ax.axvline(np.log10(bls_periods[1]), color="C1", lw=5, alpha=0.8)
ax.axvline(np.log10(bls_periods[2]), color="C1", lw=5, alpha=0.8)
ax.axvline(np.log10(bls_periods[3]), color="C1", lw=5, alpha=0.8)
ax.plot(np.log10(bls_power0.period), bls_power0.power, "k")
ax.annotate("periods = {0:.4f}, {1:.4f}, {2:.4f}, {3:.4f} d".format(*bls_periods),
            (0, 1), xycoords="axes fraction",
            xytext=(5, -5), textcoords="offset points",
            va="top", ha="left", fontsize=12)
ax.set_ylabel("bls power")
ax.set_yticks([])
ax.set_xlim(np.log10(period_grid.min()), np.log10(period_grid.max()))
ax.set_xlabel("log10(period)")

# Plot the folded transits
for i in range(2):
    ax = axes[i+1]
    x_fold = (tt_ - bls_t0s[i] + 0.5*bls_periods[i])%bls_periods[i] - 0.5*bls_periods[i]
    m = np.abs(x_fold) < 0.4
    ax.plot(x_fold[m], ff_[m], ".k")
    inds = np.argsort(x_fold[m])
    ax.plot(x_fold[m][inds], bls.model(tt_[m], bls_periods[i], bls_durations[i], bls_t0s[i])[inds])

    # Overplot the phase binned light curve
    bins = np.linspace(-0.11, 0.11, 52)
    denom, _ = np.histogram(x_fold, bins)
    num, _ = np.histogram(x_fold, bins, weights=ff_)
    denom[num == 0] = 1.0
#    ax.plot(0.5*(bins[1:] + bins[:-1]), num / denom, color="C1")

    ax.set_xlim(-0.5, 0.5)
    ax.set_ylabel("de-trended flux [ppt]")
    ax.set_xlabel("time since transit");

In [ ]:
%matplotlib inline
fig = plt.subplots(figsize=(20, 10))

plt.plot(t0+time_detrend_, flux_detrend_, 'ko-')
plt.xlim(1578., 1580.)
#plt.ylim(0.995,1.01)

In [ ]:
1.5*np.sqrt(0.002)